In [2]:

class Term:
    def __str__(self):
        pass

    def is_var(self):
        return False
    
    def is_lambda(self):
        return False

    def is_app(self):
        return False

class Var(Term):
    def __init__(self, var: str):
        self.var = var

    def __str__(self):
        return self.var

    def is_var(self):
        return True

class Lambda(Term):
    def __init__(self, var: str, t: Term):
        self.var = var
        self.t = t

    def __str__(self):
        return f"λ{self.var}. {self.t}"

    def is_lambda(self):
        return True


class App(Term):
    def __init__(self, t1: Term, t2: Term):
        self.t1 = t1
        self.t2 = t2

    def __str__(self):
        res = ""
        if (self.t1.is_lambda()):
            res += f"({self.t1})"
        else: res += f"{self.t1}"

        if (self.t2.is_var()):
            res += f" {self.t2}"
        else:
            res += f" ({self.t2})"

        return res

    def is_app(self):
        return True


x = Var("x")
lam = Lambda("x", x)
y = Var("y")
lam2 = Lambda("y", y)
app = App(lam, lam2)

print(app)

print(isinstance(None, App))

(λx. x) (λy. y)
False


In [5]:
import re

def is_var(token):
    return re.match("^[a-z]+$", token)

def is_lambda(token):
    return re.match("^lambda [a-z]+[.]$", token)

def is_term(token):
    return isinstance(token, Term)

def lambda_var(token):
    return token[7:-1]

def parse_brackets(stack):
    res = None
    while (len(stack) > 0 and stack[-1] != "("):
        token = stack.pop()
        
        if (is_term(token)):
            if res == None:
                res = token
            else:
                res = App(token, res)
        elif is_var(token):
            if res == None:
                res = Var(token)
            else:
                res = App(Var(token), res)
        elif is_lambda(token):
            lam_var = lambda_var(token)
            res = Lambda(Var(lam_var), res)

    if (len(stack) > 0):
        stack.pop()

    stack.append(res)

    return stack

def parse_term(term_str: str):
    term_str = ' '.join(term_str.split())

    regex = "lambda [a-z]+[.]|[a-z]+|[(]|[)]"

    tokens = re.findall(regex, term_str)
    stack = []

    for i, token in enumerate(tokens):
        if token == ")":
            stack = parse_brackets(stack)
        elif is_var(token):
            if len(stack) == 0:
                stack.append(token)
            elif is_term(stack[-1]):
                t = stack.pop()
                app = App(t, Var(token))
                stack.append(app)
            elif is_var(stack[-1]):
                var = stack.pop()
                app = App(Var(var), Var(token))
                stack.append(app)
            else:
                stack.append(token)
        else:
            stack.append(token)


        while len(stack) >= 2 and is_term(stack[-1]) and is_term(stack[-2]):
            t2 = stack.pop()
            t1 = stack.pop()

            stack.append(App(t1, t2))

    stack = parse_brackets(stack)

    return stack[0]

(λx. x) ((λx. x) (λz. (λx. x) z))
